# GRF Linear Operator Simulation

This notebook simulates the forward computation logic for Gaussian Random Field (GRF) linear operators using sparse step matrices wrapped in SparseLinearOperator.

In [59]:
import torch
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sys
import os

sys.path.append('/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/efficient_graph_gp_sparse/utils_sparse')

from sparse_lo import SparseLinearOperator
from linear_operator.operators import ZeroLinearOperator

torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

Libraries imported successfully!


In [60]:
def create_random_step_matrices(num_nodes, num_steps, sparsity=0.1):
    step_matrices = []
    
    for step in range(num_steps):
        random_matrix = sp.random(num_nodes, num_nodes, density=sparsity, format='csr', dtype=np.float32)
        
        crow_indices = torch.from_numpy(random_matrix.indptr).long()
        col_indices = torch.from_numpy(random_matrix.indices).long()
        values = torch.from_numpy(random_matrix.data).float()
        
        sparse_tensor = torch.sparse_csr_tensor(
            crow_indices, col_indices, values,
            (num_nodes, num_nodes),
            dtype=torch.float32
        )
        
        sparse_lo = SparseLinearOperator(sparse_tensor)
        step_matrices.append(sparse_lo)
        
    return step_matrices

num_nodes = 100
num_steps = 5
step_matrices = create_random_step_matrices(num_nodes, num_steps, sparsity=0.05)

print(f"Created {len(step_matrices)} step matrices")
print(f"Each matrix has shape: {step_matrices[0].shape}")
print(f"Number of non-zeros in first matrix: {step_matrices[0].sparse_csr_tensor._nnz()}")

Created 5 step matrices
Each matrix has shape: torch.Size([100, 100])
Number of non-zeros in first matrix: 500


In [ ]:
def K(step_matrices, modulator_vector, x1_indices=None, x2_indices=None):
    """
    Implements: K[x1, x2] @ rhs where K = Phi @ Phi^T
    K[x1, x2] @ rhs = Phi[x1] @ (Phi[x2]^T @ rhs)
    
    Args:
        step_matrices: List of SparseLinearOperator matrices
        modulator_vector: Tensor of modulation weights for each step
        x1_indices: Optional indices for rows (if None, uses all indices)
        x2_indices: Optional indices for columns (if None, uses all indices)
    """
    
    # Build the combined matrix Phi = sum(modulator_vector[i] * step_matrices[i])
    phi = ZeroLinearOperator(step_matrices[0].shape, dtype=torch.float32)
    for step, matrix in enumerate(step_matrices):
        phi += modulator_vector[step] * matrix
    
    # Handle indexing
    if x1_indices is not None:
        x1_indices = x1_indices.long().flatten()
        phi_x1 = phi[x1_indices]
    else:
        phi_x1 = phi
        
    if x2_indices is not None:
        x2_indices = x2_indices.long().flatten()
        phi_x2 = phi[x2_indices]
    else:
        phi_x2 = phi
        
    # Return K[x1, x2] = Phi[x1, :] @ Phi[x2, :]^T
    return phi_x1 @ phi_x2.transpose(-1, -2)

In [76]:
# Test the K function
print("Testing K function...")

modulator_vector = torch.randn(num_steps)
print(f"Modulator vector: {modulator_vector}")
num_nodes = step_matrices[0].shape[0]

input_vector = torch.randn(5, 1)
x1_indices = torch.tensor([0, 1, 2, 3, 4])
x2_indices = torch.tensor([5, 6, 7, 8, 9])

K_op = K(step_matrices, modulator_vector, x1_indices, x2_indices)

print(K_op)
print(f"K operator shape: {K_op.shape}")

output_vector = K_op @ input_vector
print(f"Output vector shape: {output_vector.shape}")

Testing K function...
Modulator vector: tensor([-0.1195,  1.2648, -1.6485, -0.1750,  0.4417])
Phi_x1 shape: torch.Size([5, 100]), Phi_x2 shape: torch.Size([5, 100])
K operator shape: torch.Size([5, 5])
Output vector shape: torch.Size([5, 1])


In [75]:
# test autograd
modulator_vector_grad = torch.randn(num_steps, requires_grad=True)
input_vector_grad = torch.randn(num_nodes, 1, requires_grad=True)
kernel_grad = K(step_matrices, modulator_vector_grad)
output = kernel_grad @ input_vector_grad
print(f"Output shape: {output.shape}")


loss = output.sum()
loss.backward()
print("Gradient of modulator vector:", modulator_vector_grad.grad)
print("Gradient of input vector:", input_vector_grad.grad)


Phi_x1 shape: torch.Size([100, 100]), Phi_x2 shape: torch.Size([100, 100])
Output shape: torch.Size([100, 1])
Gradient of modulator vector: tensor([ -50.3703, -106.5888,   29.8373, -565.9231, -105.3595])
Gradient of input vector: tensor([[213.3231],
        [304.5580],
        [121.4902],
        [335.4865],
        [188.4154],
        [219.2201],
        [175.8240],
        [252.9768],
        [303.3167],
        [150.0223],
        [233.3055],
        [197.1581],
        [222.4425],
        [204.3545],
        [253.2337],
        [212.1696],
        [140.0956],
        [278.9436],
        [136.3942],
        [300.7235],
        [112.4979],
        [302.6686],
        [230.2810],
        [309.7851],
        [195.4440],
        [ 28.1459],
        [195.8919],
        [250.2012],
        [312.8378],
        [289.4763],
        [186.6042],
        [365.1399],
        [101.5737],
        [270.0784],
        [306.9365],
        [208.4752],
        [ 56.1491],
        [203.6449],
        [2